# PowerShell function to force failover to an asynchronous Availability Group replica with no data loss
### The following PowerShell function will switch an asynchronous replica to synchronous.  Once the replica is synchronized, the Availability Group will failover to that secondary replica.

In [0]:
<#
.SYNOPSIS
  Name: StartFailoverToAsyncReplica.ps1
  The purpose of this script is to fail over to an asynchronous replica.
  
.DESCRIPTION
  The purpose of this script is to fail over to an asynchronous replica.  The script switches the async replica to sync.  Once the secondary is in a synchronized state,
  the Availability Group is failed over.
.PARAMETER primary
  The current primary replica for the Availability Group.

.PARAMETER asyncsecondary
  The async secondary replica you want to fail over to.

.PARAMETER agname
  The Availability Group being failed over.
      
.NOTES
    Updated: 2019-12-03       Initial build.
    Release Date: TBD
   
  Author: Frank Gill, Concurrency, Inc.

.EXAMPLE
  Failover to async replica with no data loss.
  Start-FailoverToAsyncReplica -primary replica1 -secondary replica2 -agname yourag

#>
Function Start-FailoverToAsyncReplica{
    [CmdletBinding()]
  
      PARAM ( 
          [Parameter(Mandatory=$true)]
          [string]
          $primary,
          [Parameter(Mandatory=$true)]
          [string]
          $asyncsecondary,
          [Parameter(Mandatory=$true)]
          [string]
          $agname
      )
    
    Begin{
      Write-Host "Start Start-FailoverToAsyncReplica function..."
    }
    
    Process{
      Try{
          
          <# Get a list of replicas for the Availability Group to use to resume movement after failover #>
          [System.Collections.ArrayList]$replicas = (Get-DbaAvailabilityGroup -SqlInstance $primary).AvailabilityReplicas.Name;
          <# Remove the new primary from the replica list #>
          $replicas.Remove($asyncsecondary);
  
          <# Get the synchronization state of the async secondary for use in the while loop below #>
          $syncstate = Get-DbaAgReplica -SqlInstance $asyncsecondary -AvailabilityGroup $agname | Where-Object -Property Name -EQ $asyncsecondary | Select-Object -ExpandProperty RollupSynchronizationState;
  
          <# Set the synchronization state for the async secondary to Synchronous #>
          Set-DbaAgReplica -SqlInstance $primary -AvailabilityGroup $agname -Replica $asyncsecondary -AvailabilityMode SynchronousCommit;
  
          <# Check the $syncstate variable until it flips to Synchronizing, indicating the failover can occur without data loss #>
          while($syncstate -eq "Synchronizing")
          {
              $syncstate = Get-DbaAgReplica -SqlInstance $asyncsecondary -AvailabilityGroup $agname | Where-Object -Property Name -EQ $asyncsecondary | Select-Object -ExpandProperty RollupSynchronizationState;
              Write-Host $syncstate -ForegroundColor Yellow;
          }
  
          <# Fail the Availability Group over to the formerly asynchronous secondary #>
          Invoke-DbaAgFailover -SqlInstance $asyncsecondary -AvailabilityGroup $agname -Force;
  
          <# Resume data movement for the new secondary replicas, in case it is suspended #>
          Get-DbaAgDatabase -SqlInstance $replicas | Resume-DbaAgDbDataMovement -Confirm:$false        
      }
      
      Catch{
        "Something went wrong.: $($PSItem.ToString())"
        Break
      }
  
    }
    
    End{
      If($?){ # only execute if the function was successful.
        Write-Host "Completed Start-FailoverToAsyncReplica function." -ForegroundColor Yellow;
      }
    }
  }

# Execute the function
### Once the function is created, execute it using the command below.  Because the failover is being run with the -Force parameter, data movement will be suspended on the seconday replicas.  Set -resume to Y to resume data movement for all secondary databases.

In [0]:
Start-FailoverToAsyncReplica -primary FBGSQL2019VM1 -asyncsecondary FBGSQL2019VM3 -agname FBGSQL2019AG -resume Y;

# **Note: If the AG replica entered as primary is not a primary replica, you will see the following error:**

In [0]:
WARNING: [22:59:33][Set-DbaAgReplica] Failure | Availability group 'yourreplica' cannot process the ALTER AVAILABILITY GROUP command, because the local availability replica is not t
he primary replica.  Connect to the server instance that is currently hosting the primary replica of this availability group, and rerun the command.
WARNING: [22:59:33][Invoke-DbaAgFailover] Failure | Cannot failover availability group 'yourreplica' to this instance of SQL Server.  The local availability replica is already the p
rimary replica of the availability group.  To failover this availability group to another instance of SQL Server, run the failover command on that instance of SQL Server.  If local i
nstance of SQL Server is intended to host the primary replica of the availability group, then no action is required.

# **If you see this message, confirm your primary replica and execute the function again.**